Data loading:

In [2]:
label_set = ['Negative', 'Neutral', 'Positive']
STR_TO_NUM = {k: i for i, k in enumerate(label_set)}
NUM_TO_STR = {i:k for i, k in enumerate(label_set)}

import pandas as pd
df = pd.read_json("bcs_polsent_007.jsonl", orient="records", lines=True)
df["label"] = df.label.apply(lambda s: STR_TO_NUM[s])
df = df[["sentence", "label", "split"]].rename(columns={"sentence": "text", "label":"labels"})
train = df[df.split=="train"].drop(columns=["split"])
dev = df[df.split=="dev"].drop(columns=["split"])
test = df[df.split=="test"].drop(columns=["split"])

Define functions for training and evaluating

In [3]:
def train_model(train_df, model_name):
    if model_name == "EMBEDDIA/crosloengual-bert":
        model_type = "bert"
        NUM_EPOCH = 3
    elif model_name == "classla/bcms-bertic":
        model_type = "electra"
        NUM_EPOCH = 9
    elif model_name == "xlm-roberta-base":
        model_type = "xlmroberta"
        NUM_EPOCH = 40
    else:
        raise AttributeError(f"Expected either xlm-roberta-base, classla/bcms-bertic, or EMBEDDIA/crosloengual-bert, got {model_name}.")

    from simpletransformers.classification import ClassificationModel
    import torch
    torch.cuda.empty_cache()
    model_args = {
        "num_train_epochs": NUM_EPOCH,
        "learning_rate": 4e-5,
        "overwrite_output_dir": True,
        "train_batch_size": 8, 
        "no_save": True,
        "no_cache": True,
        "overwrite_output_dir": True,
        "save_steps": -1,
        "max_seq_length": 512,
        "silent": True,
    }

    model = ClassificationModel(
        model_type, model_name, num_labels=3, use_cuda=True, args=model_args
    )
    model.train_model(train_df)
    return model


def eval_model(model, test_df):
    y_true_enc = test_df.labels
    from tqdm.auto import tqdm
    y_pred_enc = [model.predict(i)[0][0] for i in tqdm(test_df.text.values)]

    y_true = [NUM_TO_STR[i] for i in y_true_enc]
    y_pred = [NUM_TO_STR[i] for i in y_pred_enc]
    from sklearn.metrics import f1_score
    macroF1 = f1_score(y_true, y_pred, labels=label_set, average="macro")

    return {"macroF1": macroF1, "y_true": y_true, "y_pred": y_pred}


In [4]:
models = "xlm-roberta-base classla/bcms-bertic EMBEDDIA/crosloengual-bert".split()
# models = ["EMBEDDIA/crosloengual-bert"]
from tqdm.auto import tqdm
for model_name in tqdm(models*6):
    # Full train-test:
    model = train_model(train, model_name)
    df = pd.read_json("bcs_polsent_007.jsonl", orient="records", lines=True)
    df["label"] = df.label.apply(lambda s: STR_TO_NUM[s])
    df = df[["sentence", "label", "split", "country"]].rename(columns={"sentence": "text", "label":"labels"})
    train = df[df.split=="train"].drop(columns=["split", "country"])
    condition_is_test = df.split=="test"
    test = df[condition_is_test].drop(columns=["split", "country"])
    stats = eval_model(model, test)
    stats["train_split"] = "train"
    stats["eval_split"] = "test"
    stats["model_name"] = model_name
    with open("007_results.jsonl", "a") as f:
        f.write(f"{stats}\n")

    condition_is_dev = df.split=="dev"
    dev = df[condition_is_dev].drop(columns=["split", "country"])
    stats = eval_model(model, dev)
    stats["train_split"] = "train"
    stats["eval_split"] = "dev"
    stats["model_name"] = model_name
    with open("007_results.jsonl", "a") as f:
        f.write(f"{stats}\n")
        
    # Eval on specific countries
    for country in ["HR", "SRB"]:
        import pandas as pd
        df = pd.read_json("bcs_polsent_007.jsonl", orient="records", lines=True)
        df["label"] = df.label.apply(lambda s: STR_TO_NUM[s])
        df = df[["sentence", "label", "split", "country"]].rename(columns={"sentence": "text", "label":"labels"})
        condition_is_test = df.split=="test"
        condition_is_lang = df.country == country
        test = df[condition_is_test & condition_is_lang].drop(columns=["split", "country"])
        stats = eval_model(model, test)
        stats["train_split"] = "train"
        stats["eval_split"] = f"test_{country}"
        stats["model_name"] = model_name
        with open("007_results.jsonl", "a") as f:
            f.write(f"{stats}\n")

        condition_is_dev = df.split=="dev"
        condition_is_lang = df.country == country
        dev = df[condition_is_dev & condition_is_lang].drop(columns=["split", "country"])
        stats = eval_model(model, dev)
        stats["train_split"] = "train"
        stats["eval_split"] = f"dev_{country}"
        stats["model_name"] = model_name
        with open("007_results.jsonl", "a") as f:
            f.write(f"{stats}\n")

  0%|          | 0/18 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

...

  0%|          | 0/82 [00:00<?, ?it/s]

  0%|          | 0/119 [00:00<?, ?it/s]

  0%|          | 0/59 [00:00<?, ?it/s]

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense

RuntimeError: CUDA out of memory. Tried to allocate 96.00 MiB (GPU 0; 31.75 GiB total capacity; 8.78 GiB already allocated; 31.75 MiB free; 8.92 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
# models = "xlm-roberta-base classla/bcms-bertic EMBEDDIA/crosloengual-bert".split()
# from tqdm.auto import tqdm
# for model_name in tqdm(models*5):
#     model = train_model(train, model_name)
#     stats = eval_model(model, test)
#     stats["train_split"] = "train"
#     stats["eval_split"] = "test"
#     stats["model_name"] = model_name
#     with open("007_results.jsonl", "a") as f:
#         f.write(f"{stats}\n")

# Train on train, evaluate on specific country

This was done already in the cell above.

In [ ]:
# models = "xlm-roberta-base classla/bcms-bertic EMBEDDIA/crosloengual-bert".split()
# from tqdm.auto import tqdm
# for model_name in tqdm(models*6):
#     model = train_model(train, model_name)
#     for country in ["HR", "SRB"]:
#         import pandas as pd
#         df = pd.read_json("bcs_polsent_007.jsonl", orient="records", lines=True)
#         df["label"] = df.label.apply(lambda s: STR_TO_NUM[s])
#         df = df[["sentence", "label", "split", "country"]].rename(columns={"sentence": "text", "label":"labels"})
#         condition_is_test = df.split=="test"
#         condition_is_lang = df.country == country
#         test = df[condition_is_test & condition_is_lang].drop(columns=["split", "country"])
#         stats = eval_model(model, test)
#         stats["train_split"] = "train"
#         stats["eval_split"] = f"test_{country}"
#         stats["model_name"] = model_name
#         with open("007_results.jsonl", "a") as f:
#             f.write(f"{stats}\n")

# Train on specific country, evaluate on specific country:

The implementation is less than elegant (by light years), but this was written when the training was already underway and reformatting would require restarting.

In [ ]:
models = "xlm-roberta-base classla/bcms-bertic EMBEDDIA/crosloengual-bert".split()
# models = ["EMBEDDIA/crosloengual-bert"]
from tqdm.auto import tqdm
for model_name in tqdm(models*6):
    for country_train in ["HR", "SRB"]:
        df = pd.read_json("bcs_polsent_007.jsonl", orient="records", lines=True)
        df["label"] = df.label.apply(lambda s: STR_TO_NUM[s])
        df = df[["sentence", "label", "split", "country"]].rename(columns={"sentence": "text", "label":"labels"})
        condition_is_train = df.split=="train"
        condition_is_lang = df.country == country_train
        train = df[condition_is_train & condition_is_lang].drop(columns=["split", "country"])
        model = train_model(train, model_name)
        for country_eval in ["HR", "SRB"]:
            df = pd.read_json("bcs_polsent_007.jsonl", orient="records", lines=True)
            df["label"] = df.label.apply(lambda s: STR_TO_NUM[s])
            df = df[["sentence", "label", "split", "country"]].rename(columns={"sentence": "text", "label":"labels"})
            condition_is_test = df.split=="test"
            condition_is_lang = df.country == country_eval
            test = df[condition_is_test & condition_is_lang].drop(columns=["split", "country"])
            stats = eval_model(model, test)
            stats["train_split"] = f"train_{country_train}"
            stats["eval_split"] = f"test_{country_eval}"
            stats["model_name"] = model_name
            with open("007_results.jsonl", "a") as f:
                f.write(f"{stats}\n")

            df = pd.read_json("bcs_polsent_007.jsonl", orient="records", lines=True)
            df["label"] = df.label.apply(lambda s: STR_TO_NUM[s])
            df = df[["sentence", "label", "split", "country"]].rename(columns={"sentence": "text", "label":"labels"})
            condition_is_dev = df.split=="dev"
            condition_is_lang = df.country == country_eval
            dev = df[condition_is_dev & condition_is_lang].drop(columns=["split", "country"])
            stats = eval_model(model, dev)
            stats["train_split"] = f"train_{country_train}"
            stats["eval_split"] = f"dev_{country_eval}"
            stats["model_name"] = model_name
            with open("007_results.jsonl", "a") as f:
                f.write(f"{stats}\n")